# 1. Importação das bibliotecas, definição dos métodos e tratamento inicial




In [ ]:
%%capture
!pip install ray
!pip install unicode
!pip install unidecode
!pip install boruta
!pip install catboost

In [ ]:
import pandas as pd
import ray
import plotly.express as px
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import numpy as np
import unidecode
from scipy import stats
from google.colab import drive
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def find_best_lambda(data, lambda_range):
    best_lambda = None
    best_llf = -np.inf  # initialize with negative infinity

    for lmbda in lambda_range:
        llf = stats.yeojohnson_llf(lmbda, data)
        if llf > best_llf:
            best_llf = llf
            best_lambda = lmbda

    return best_lambda

In [ ]:
def remove_accents(a):
  a = a.encode('utf-8', errors='ignore')
  return unidecode.unidecode(a.decode('utf-8'))

In [ ]:
cadunico = pd.read_csv("/content/drive/MyDrive/Hunger prediction/data/Brazil/cadunico_data_brazil.csv")
sisvan = pd.read_excel("/content/drive/MyDrive/Hunger prediction/data/Brazil/altura_idade.xlsx")

In [ ]:
sisvan

,ALTURA X IDADE,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Região,Código UF,UF,Código IBGE,Município,Altura Muito Baixa\npara a Idade,NaN,Altura Baixa\npara a Idade,NaN,Altura Adequada\npara a Idade,NaN,Total
1,NaN,NaN,NaN,NaN,NaN,Quantidade,%,Quantidade,%,Quantidade,%,NaN
2,CENTRO-OESTE,53,DF,530010,BRASILIA,2344,3.06%,4438,5.79%,69802,91.14%,76584
3,TOTAL ESTADO DISTRITO FEDERAL,NaN,NaN,NaN,NaN,2344,3.06%,4438,5.79%,69802,91.14%,76584
4,CENTRO-OESTE,52,GO,520005,ABADIA DE GOIAS,15,4.36%,16,4.65%,313,90.99%,344
...,...,...,...,...,...,...,...,...,...,...,...,...
5604,TOTAL BRASIL,NaN,NaN,NaN,NaN,311693,4.74%,427905,6.5%,5841819,88.76%,6581417
5605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5607,VersÃ£o 3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sisvan[sisvan.columns[6]][0] = 'Altura Muito Baixa para a Idade (%)'
sisvan[sisvan.columns[8]][0] = 'Altura Baixa para a Idade (%)'
sisvan[sisvan.columns[10]][0] = 'Altura Adequada para a Idade (%)'

sisvan.dropna(inplace=True)

sisvan.columns = sisvan.iloc[0]
sisvan = sisvan[1:]
sisvan.reset_index(drop=True, inplace=True)

for i, val in enumerate(sisvan['Município']):
  val += ' - '
  val += sisvan['UF'][i]
  sisvan['Município'][i] = val

sisvan = sisvan.iloc[:, [4, 6, 8]]

sisvan.replace(['%'], [''], regex=True, inplace=True)
sisvan['Município'] = sisvan['Município'].apply(remove_accents)

for col in sisvan.columns[1:]:
  sisvan[col].replace(['-'], [0], regex=True, inplace=True)
  sisvan[col] = sisvan[col].astype(float)

sisvan['Município'] = sisvan['Município'].str.lower()

sisvan['Altura/Idade < Escore-z -2 (%)'] = sisvan[sisvan.columns[1]] + sisvan[sisvan.columns[2]]
sisvan.drop(columns=sisvan.columns[1:3], inplace=True)

<ipython-input-8-fe9856b18186>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sisvan['Município'][i] = val


In [ ]:
cadunico['Município'] = cadunico['Município'].str.lower()
cadunico['Município'] = cadunico['Município'].apply(remove_accents)

In [ ]:
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', None)
# cadunico

In [ ]:
df = sisvan.join(cadunico.set_index('Município'), on='Município')

In [ ]:

mask = df['Município'].str.contains(' - ce')
df_test = df[mask]
df_train = df[~mask]

In [ ]:
df_test

,Município,Altura/Idade < Escore-z -2 (%),Estado cadastral da família / Sem Registro Civil,Estado cadastral da família / Cadastrado,Faixa da renda total da família / Até 1 S.M.,Faixa da renda total da família / Entre 1 e 2 S.M.,Faixa da renda total da família / Entre 2 e 3 S.M.,Faixa da renda total da família / Acima de 3 S.M.,Faixa da renda familiar per capita / Pobreza 1 (até R$ 109),Faixa da renda familiar per capita / Pobreza 2 (de R$ 109 a R$ 218),Faixa da renda familiar per capita / Baixa Renda,Faixa da renda familiar per capita / Acima de 1/2 S.M.,Forma de coleta / Informação migrada como inexistente,Forma de coleta / Sem visita Domiciliar,Forma de coleta / Com visita domiciliar,Número de meses após a última atualização cadastral / até 12 Meses,Número de meses após a última atualização cadastral / 13 a 18 Meses,Número de meses após a última atualização cadastral / 19 a 24 Meses,Número de meses após a última atualização cadastral / 25 a 36 Meses,Número de meses após a última atualização cadastral / 37 a 48 Meses,Número de meses após a última atualização cadastral / acima de 48 Meses,Recebe PBF família / Não,Recebe PBF família / Sim,Calçamento em frente ao seu domicílio / Total,Calçamento em frente ao seu domicílio / Parcial,Calçamento em frente ao seu domicílio / Não existe,Situação do domicílio / Urbanas,Situação do domicílio / Rurais,Espécie do domicílio / Particular Permanente,Espécie do domicílio / Particular improvisado,Espécie do domicílio / Coletivo,Existência de banheiro / Sim,Existência de banheiro / Não,Forma de abastecimento de água / Rede geral de distribuição,Forma de abastecimento de água / Poço ou nascente,Forma de abastecimento de água / Cisterna,Forma de abastecimento de água / Outra forma,Forma de coleta do lixo / É coletado diretamente,Forma de coleta do lixo / É coletado indiretamente,Forma de coleta do lixo / É queimado ou enterrado na propriedade,"Forma de coleta do lixo / É jogado em terreno baldio ou logradouro (rua, avenida, etc.)",Forma de coleta do lixo / É jogado em rio ou mar,Forma de coleta do lixo / Tem outro destino,Forma de escoamento sanitário / Rede coletora de esgoto ou pluvial,Forma de escoamento sanitário / Fossa séptica,Forma de escoamento sanitário / Fossa rudimentar,Forma de escoamento sanitário / Vala a céu aberto,"Forma de escoamento sanitário / Direto para um rio, lago ou mar",Forma de escoamento sanitário / Outra forma,Material predominante nas paredes externas do domicílio / Alvenaria/tijolo com revestimento,Material predominante nas paredes externas do domicílio / Alvenaria/tijolo sem revestimento,Material predominante nas paredes externas do domicílio / Madeira aparelhada,Material predominante nas paredes externas do domicílio / Taipa revestida,Material predominante nas paredes externas do domicílio / Taipa não revestida,Material predominante nas paredes externas do domicílio / Madeira aproveitada,Material predominante nas paredes externas do domicílio / Palha,Material predominante nas paredes externas do domicílio / Outro Material,Material predominante no piso do domicílio / Terra,Material predominante no piso do domicílio / Cimento,Material predominante no piso do domicílio / Madeira aproveitada,Material predominante no piso do domicílio / Madeira aparelhada,"Material predominante no piso do domicílio / Cerâmica, lajota ou pedra",Material predominante no piso do domicílio / Carpete,Material predominante no piso do domicílio / Outro Material,Água canalizada no domicílio / Sim,Água canalizada no domicílio / Não,Tipo de iluminação / Elétrica com medidor próprio,Tipo de iluminação / Elétrica com medidor comunitário,Tipo de iluminação / Elétrica sem medidor,"Tipo de iluminação / Óleo, querosene ou gás",Tipo de iluminação / Vela,Tipo de iluminação / Outra forma,Família Indígena / Sim,Família Indígena / Não,Família quilombola / Sim,Família quilombola / Não,Cor ou raça / Branca,Cor ou raça / Preta,Cor ou raça / Amarela,Cor ou raça / Parda,Cor ou raça / Indígena,Estado cadastral d

In [ ]:
x_train = df_train.drop(columns=df_train.columns[0:2])
y_train = df_train[df_train.columns[1]]

x_test = df_test.drop(columns=df_test.columns[0:2])
y_test = df_test[df_test.columns[1]]

# 2. Teste dos algoritmos

In [ ]:
%%capture
!pip install BorutaShap
!pip install xgboost
!pip install lightgbm

In [ ]:
import catboost as cb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from BorutaShap import BorutaShap, load_data
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
from  sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

## 2.1 Random Forest

In [ ]:
forest = RandomForestRegressor(
   n_jobs = -1, max_depth = 7, n_estimators=200
)

Feature_Selector = BorutaShap(model=forest,
                              importance_measure='shap',
                              classification=False)

Feature_Selector.fit(X=x_train, y=y_train, n_trials=100, random_state=0)

  0%|          | 0/100 [00:00<?, ?it/s]

37 attributes confirmed important: ['Recebe PBF família / Não.1', 'Recebe PBF família / Sim', 'Recebe PBF família / Sim.1', 'Cor ou raça / Parda', 'A pessoa concluiu o curso? / Não', 'Último ano e série do curso que a pessoa frequentou / Quinto(a)', 'Faixa da renda total da família / Entre 2 e 3 S.M.', 'Existência de banheiro / Não', 'Relação de parentesco com o responsável familiar / Cônjuge ou companheiro(a)', 'Faixa etária / Entre 0 e 4', 'Faixa etária / Entre 18 a 24', 'Tipo de iluminação / Elétrica sem medidor', 'Pessoa frequenta escola / Não, já frequentou', 'Cor ou raça / Branca', 'Ano e série do curso que a pessoa frequenta / Segundo(a)', 'Material predominante nas paredes externas do domicílio / Taipa não revestida', 'Faixa etária / Entre 16 a 17', 'Relação de parentesco com o responsável familiar / Pessoa Responsável pela Unidade Familiar - RF', 'Faixa etária / Entre 60 a 64', 'Faixa etária / Entre 7 a 15', 'Água canalizada no domicílio / Não', 'Faixa da renda familiar per ca

In [ ]:
accepted = Feature_Selector.Subset(tentative=True).columns
accepted

Index(['Recebe PBF família / Não.1', 'Recebe PBF família / Sim',
       'Recebe PBF família / Sim.1', 'Cor ou raça / Parda',
       'A pessoa concluiu o curso? / Não',
       'Último ano e série do curso que a pessoa frequentou / Quinto(a)',
       'Faixa da renda total da família / Entre 2 e 3 S.M.',
       'Existência de banheiro / Não',
       'Relação de parentesco com o responsável familiar / Cônjuge ou companheiro(a)',
       'Faixa etária / Entre 0 e 4', 'Faixa etária / Entre 18 a 24',
       'Tipo de iluminação / Elétrica sem medidor',
       'Pessoa frequenta escola / Não, já frequentou', 'Cor ou raça / Branca',
       'Ano e série do curso que a pessoa frequenta / Segundo(a)',
       'Material predominante nas paredes externas do domicílio / Taipa não revestida',
       'Faixa etária / Entre 16 a 17',
       'Relação de parentesco com o responsável familiar / Pessoa Responsável pela Unidade Familiar - RF',
       'Faixa etária / Entre 60 a 64', 'Faixa etária / Entre 7 a 15',


In [ ]:
accepted = ['Recebe PBF família / Não.1', 'Recebe PBF família / Sim',
       'Recebe PBF família / Sim.1', 'Cor ou raça / Parda',
       'A pessoa concluiu o curso? / Não',
       'Último ano e série do curso que a pessoa frequentou / Quinto(a)',
       'Faixa da renda total da família / Entre 2 e 3 S.M.',
       'Existência de banheiro / Não',
       'Relação de parentesco com o responsável familiar / Cônjuge ou companheiro(a)',
       'Faixa etária / Entre 0 e 4', 'Faixa etária / Entre 18 a 24',
       'Tipo de iluminação / Elétrica sem medidor',
       'Pessoa frequenta escola / Não, já frequentou', 'Cor ou raça / Branca',
       'Ano e série do curso que a pessoa frequenta / Segundo(a)',
       'Material predominante nas paredes externas do domicílio / Taipa não revestida',
       'Faixa etária / Entre 16 a 17',
       'Relação de parentesco com o responsável familiar / Pessoa Responsável pela Unidade Familiar - RF',
       'Faixa etária / Entre 60 a 64', 'Faixa etária / Entre 7 a 15',
       'Água canalizada no domicílio / Não',
       'Faixa da renda familiar per capita / Acima de 1/2 S.M.',
       'Faixa etária / Entre 35 a 39', 'Recebe PBF família / Não',
       'Faixa etária / Entre 45 a 49',
       'Material predominante no piso do domicílio / Terra',
       'Faixa etária / Entre 55 a 59', 'Água canalizada no domicílio / Sim',
       'Faixa da renda total da família / Acima de 3 S.M.',
       'Grupos Populacionais Tradicionais e Específicos / Familia Extrativista',
       'Faixa da renda total da família / Entre 1 e 2 S.M.',
       'Faixa da renda total da família / Até 1 S.M.',
       'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo com revestimento',
       'Material predominante nas paredes externas do domicílio / Palha',
       'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo sem revestimento',
       'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos)',
       'Relação de parentesco com o responsável familiar / Filho(a)',
       'Último ano e série do curso que a pessoa frequentou / Sexto(a)',
       'Material predominante no piso do domicílio / Cerâmica, lajota ou pedra',
       'Forma de escoamento sanitário / Vala a céu aberto']

In [ ]:
x_tr = x_train[accepted]

In [ ]:
grid = {'max_depth': [5, 8, 10],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 25, 50, 70],
        'max_features': ['log2', 0.25, 'sqrt', 1.0]}

clf = GridSearchCV(RandomForestRegressor(n_estimators=200), grid, cv = 10,
                   scoring='neg_root_mean_squared_error')
clf.fit(x_tr, y_train)
clf.best_params_

{'max_depth': 10,
 'max_features': 1.0,
 'min_samples_leaf': 25,
 'min_samples_split': 10}

In [ ]:
forest = RandomForestRegressor(n_estimators=200, max_depth=10, max_features=1.0,
                               min_samples_leaf=25, min_samples_split=10
)

scores = cross_validate(forest, x_tr, y_train, cv=10,
                               scoring='neg_root_mean_squared_error')

rmse = np.mean(scores['test_score']*(-1))
std = np.std(scores['test_score']*(-1))

print(rmse)
print(std)

4.879823781035269
0.5305918099371907


## 2.2 Gradient Boosting

In [ ]:
gb = GradientBoostingRegressor(
   max_depth = 7, n_estimators=200
)

Feature_Selector = BorutaShap(model=gb,
                              importance_measure='shap',
                              classification=False)

Feature_Selector.fit(X=x_train, y=y_train, n_trials=100, random_state=0)

  0%|          | 0/100 [00:00<?, ?it/s]

16 attributes confirmed important: ['Cor ou raça / Branca', 'Material predominante no piso do domicílio / Terra', 'Faixa da renda familiar per capita / Acima de 1/2 S.M.', 'Função principal / Militar ou servidor público', 'Função principal / Empregado sem carteira de trabalho assinada', 'Relação de parentesco com o responsável familiar / Cônjuge ou companheiro(a)', 'Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos)', 'Faixa etária / Entre 7 a 15', 'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo com revestimento', 'Faixa da renda total da família / Até 1 S.M.', 'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)', 'Recebe PBF família / Não', 'Faixa etária / Entre 18 a 24', 'Faixa etária / Entre 60 a 64', 'Faixa etária / Entre 35 a 39', 'Faixa etária / Entre 0 e 4']
1 attributes confirmed unimportant: ['Faixa da renda total da família / Entre 2 e 3 S.M.']
4 tentative attributes remains: ['Recebe ajuda de o

In [ ]:
accepted = Feature_Selector.Subset(tentative=True).columns
accepted

Index(['Cor ou raça / Branca',
       'Material predominante no piso do domicílio / Terra',
       'Faixa da renda familiar per capita / Acima de 1/2 S.M.',
       'Função principal / Militar ou servidor público',
       'Função principal / Empregado sem carteira de trabalho assinada',
       'Relação de parentesco com o responsável familiar / Cônjuge ou companheiro(a)',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos)',
       'Faixa etária / Entre 7 a 15',
       'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo com revestimento',
       'Faixa da renda total da família / Até 1 S.M.',
       'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)',
       'Recebe PBF família / Não', 'Faixa etária / Entre 18 a 24',
       'Faixa etária / Entre 60 a 64', 'Faixa etária / Entre 35 a 39',
       'Faixa etária / Entre 0 e 4',
       'Recebe ajuda de outra forma / Opção marcada no formulário',
       'Exis

In [ ]:
x_tr = x_train[accepted]

In [ ]:
grid = {'learning_rate': [0.025, 0.05, 0.1, 0.2, 0.3],
        'max_depth': [2, 3, 5, 7, 10],
        'max_features': ['log2', 0.25, 'sqrt', 1.0],
        'subsample': [0.15, 0.5, 0.75, 1.0]}

clf = GridSearchCV(GradientBoostingRegressor(n_estimators=200), grid, cv = 10,
                   scoring='neg_root_mean_squared_error')
clf.fit(x_tr, y_train)
clf.best_params_

In [1]:
gb = GradientBoostingRegressor(n_estimators=200, max_depth=10,
   max_features = 1, min_samples_leaf = 25, min_samples_split = 20
)

scores = cross_validate(gb, x_tr, y_train, cv=10,
                                scoring='neg_root_mean_squared_error')

rmse = np.mean(scores['test_score']*(-1))
std = np.std(scores['test_score']*(-1))

print(rmse)
print(std)

## 2.3 Catboost

In [ ]:
ctb = CatBoostRegressor(iterations=200)

# no model selected default is Random Forest, if classification is False it is a Regression problem
Feature_Selector = BorutaShap(model=ctb,
                              importance_measure='shap',
                              classification=False)

Feature_Selector.fit(X=x_train, y=y_train, n_trials=100, random_state=0)

  0%|          | 0/100 [00:00<?, ?it/s]

18 attributes confirmed important: ['Existência de banheiro / Não', 'Recebe PBF família / Não', 'Faixa etária / Entre 7 a 15', 'Faixa etária / Entre 18 a 24', 'Cor ou raça / Branca', 'Faixa etária / Entre 0 e 4', 'Material predominante no piso do domicílio / Terra', 'Curso mais elevado que a pessoa frequentou / Ensino Fundamental EJA -séries iniciais (Supletivo 1ª a 4ª)', 'Função principal / Empregado sem carteira de trabalho assinada', 'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)', 'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo com revestimento', 'Faixa etária / Entre 35 a 39', 'Função principal / Militar ou servidor público', 'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo sem revestimento', 'Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos)', 'Faixa da renda total da família / Até 1 S.M.', 'Relação de parentesco com o responsável familiar / Cônjuge ou companheiro(a)

In [ ]:
accepted = Feature_Selector.Subset(tentative=True).columns
accepted

Index(['Existência de banheiro / Não', 'Recebe PBF família / Não',
       'Faixa etária / Entre 7 a 15', 'Faixa etária / Entre 18 a 24',
       'Cor ou raça / Branca', 'Faixa etária / Entre 0 e 4',
       'Material predominante no piso do domicílio / Terra',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental EJA -séries iniciais (Supletivo 1ª a 4ª)',
       'Função principal / Empregado sem carteira de trabalho assinada',
       'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)',
       'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo com revestimento',
       'Faixa etária / Entre 35 a 39',
       'Função principal / Militar ou servidor público',
       'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo sem revestimento',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos)',
       'Faixa da renda total da família / Até 1 S.M.',
       'Relação de par

In [ ]:
accepted = ['Existência de banheiro / Não', 'Recebe PBF família / Não',
       'Faixa etária / Entre 7 a 15', 'Faixa etária / Entre 18 a 24',
       'Cor ou raça / Branca', 'Faixa etária / Entre 0 e 4',
       'Material predominante no piso do domicílio / Terra',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental EJA -séries iniciais (Supletivo 1ª a 4ª)',
       'Função principal / Empregado sem carteira de trabalho assinada',
       'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)',
       'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo com revestimento',
       'Faixa etária / Entre 35 a 39',
       'Função principal / Militar ou servidor público',
       'Material predominante nas paredes externas do domicílio / Alvenaria/tijolo sem revestimento',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos)',
       'Faixa da renda total da família / Até 1 S.M.',
       'Relação de parentesco com o responsável familiar / Cônjuge ou companheiro(a)',
       'Faixa da renda familiar per capita / Acima de 1/2 S.M.',
       'Recebe ajuda de outra forma / Opção marcada no formulário',
       'Faixa etária / Entre 60 a 64',
       'Faixa da renda total da família / Entre 2 e 3 S.M.']

In [ ]:
x_tr = x_train[accepted]

In [ ]:
grid = {'learning_rate': [0.025, 0.05, 0.1, 0.2, 0.3],
        'max_depth': [3, 6, 9],
        'leaf_estimation_iterations': [1, 10],
        'l2_leaf_reg': [1, 3, 6, 9]}

clf = GridSearchCV(CatBoostRegressor(iterations=200), grid, cv = 10,
                   scoring='neg_root_mean_squared_error')
clf.fit(x_tr, y_train)
clf.best_params_

A saída de streaming foi truncada nas últimas 5000 linhas.
0:	learn: 5.3503055	total: 2.9ms	remaining: 577ms
1:	learn: 5.1917582	total: 6ms	remaining: 594ms
2:	learn: 5.0757071	total: 9.04ms	remaining: 594ms
3:	learn: 5.0035648	total: 13.9ms	remaining: 683ms
4:	learn: 4.9537383	total: 19.9ms	remaining: 777ms
5:	learn: 4.9233706	total: 26.8ms	remaining: 866ms
6:	learn: 4.9002801	total: 36.5ms	remaining: 1.01s
7:	learn: 4.8791872	total: 39.3ms	remaining: 943ms
8:	learn: 4.8629887	total: 41.9ms	remaining: 890ms
9:	learn: 4.8486735	total: 44.7ms	remaining: 850ms
10:	learn: 4.8346681	total: 47.4ms	remaining: 815ms
11:	learn: 4.8258877	total: 50.1ms	remaining: 786ms
12:	learn: 4.8146641	total: 53.4ms	remaining: 768ms
13:	learn: 4.8059112	total: 56.2ms	remaining: 746ms
14:	learn: 4.7937643	total: 59ms	remaining: 727ms
15:	learn: 4.7866095	total: 61.7ms	remaining: 710ms
16:	learn: 4.7792393	total: 64.3ms	remaining: 693ms
17:	learn: 4.7687954	total: 67ms	remaining: 678ms
18:	learn: 4.7618262	to

{'l2_leaf_reg': 9,
 'leaf_estimation_iterations': 1,
 'learning_rate': 0.05,
 'max_depth': 6}

In [ ]:
ctb = CatBoostRegressor(n_estimators = 200,
                                l2_leaf_reg=9, leaf_estimation_iterations=1,
                                learning_rate=0.05, max_depth=6)

scores = cross_validate(ctb, x_tr, y_train, cv=10,
                               scoring='neg_root_mean_squared_error')

rmse = np.mean(scores['test_score']*(-1))
std = np.std(scores['test_score']*(-1))

print(rmse)
print(std)

0:	learn: 5.6221044	total: 64.1ms	remaining: 12.7s
1:	learn: 5.5721632	total: 79.2ms	remaining: 7.84s
2:	learn: 5.5250973	total: 91.8ms	remaining: 6.03s
3:	learn: 5.4821096	total: 108ms	remaining: 5.29s
4:	learn: 5.4407388	total: 122ms	remaining: 4.74s
5:	learn: 5.4043308	total: 135ms	remaining: 4.37s
6:	learn: 5.3696177	total: 162ms	remaining: 4.47s
7:	learn: 5.3372904	total: 192ms	remaining: 4.6s
8:	learn: 5.3067335	total: 211ms	remaining: 4.48s
9:	learn: 5.2796020	total: 228ms	remaining: 4.34s
10:	learn: 5.2524300	total: 255ms	remaining: 4.39s
11:	learn: 5.2253770	total: 271ms	remaining: 4.25s
12:	learn: 5.2027456	total: 300ms	remaining: 4.32s
13:	learn: 5.1813474	total: 317ms	remaining: 4.21s
14:	learn: 5.1623067	total: 331ms	remaining: 4.08s
15:	learn: 5.1434236	total: 346ms	remaining: 3.98s
16:	learn: 5.1257050	total: 360ms	remaining: 3.87s
17:	learn: 5.1085938	total: 373ms	remaining: 3.77s
18:	learn: 5.0919840	total: 389ms	remaining: 3.71s
19:	learn: 5.0756727	total: 403ms	remai

## 2.4 XGBoost

In [ ]:
xgb = XGBRegressor(n_estimators=200)

# no model selected default is Random Forest, if classification is False it is a Regression problem
Feature_Selector = BorutaShap(model=xgb,
                              importance_measure='shap',
                              classification=False)

Feature_Selector.fit(X=x_train, y=y_train, n_trials=100, random_state=0)

  0%|          | 0/100 [00:00<?, ?it/s]

13 attributes confirmed important: ['Curso mais elevado que a pessoa frequentou / Ensino Médio, 2º grau, Médio 2º ciclo (Científico, Clássico, Técnico, Normal)', 'Faixa etária / Entre 7 a 15', 'Faixa etária / Entre 18 a 24', 'Cor ou raça / Branca', 'Faixa etária / Entre 0 e 4', 'Material predominante no piso do domicílio / Terra', 'Função principal / Empregado sem carteira de trabalho assinada', 'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)', 'Faixa etária / Entre 35 a 39', 'Água canalizada no domicílio / Não', 'Faixa da renda total da família / Até 1 S.M.', 'Curso mais elevado que a pessoa frequentou / Ensino Fundamental EJA -séries iniciais (Supletivo 1ª a 4ª)', 'Faixa etária / Entre 55 a 59']
222 attributes confirmed unimportant: ['Existência de banheiro / Não', 'A pessoa concluiu o curso? / Não', 'Número de meses após a última atualização cadastral / 19 a 24 Meses', 'Pessoa estava afastada na semana passada? / Sim', 'Grupos Populacionais Tradicionais e

In [ ]:
accepted = Feature_Selector.Subset(tentative=True).columns
accepted

Index(['Curso mais elevado que a pessoa frequentou / Ensino Médio, 2º grau, Médio 2º ciclo (Científico, Clássico, Técnico, Normal)',
       'Faixa etária / Entre 7 a 15', 'Faixa etária / Entre 18 a 24',
       'Cor ou raça / Branca', 'Faixa etária / Entre 0 e 4',
       'Material predominante no piso do domicílio / Terra',
       'Função principal / Empregado sem carteira de trabalho assinada',
       'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)',
       'Faixa etária / Entre 35 a 39', 'Água canalizada no domicílio / Não',
       'Faixa da renda total da família / Até 1 S.M.',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental EJA -séries iniciais (Supletivo 1ª a 4ª)',
       'Faixa etária / Entre 55 a 59', 'Recebe PBF família / Sim',
       'Função principal / Militar ou servidor público',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos)',
       'Relação de parentesco com o responsável familia

In [ ]:
accepted = ['Curso mais elevado que a pessoa frequentou / Ensino Médio, 2º grau, Médio 2º ciclo (Científico, Clássico, Técnico, Normal)',
       'Faixa etária / Entre 7 a 15', 'Faixa etária / Entre 18 a 24',
       'Cor ou raça / Branca', 'Faixa etária / Entre 0 e 4',
       'Material predominante no piso do domicílio / Terra',
       'Função principal / Empregado sem carteira de trabalho assinada',
       'Curso que a pessoa frequenta / Ensino Fundamental regular (duração 9 anos)',
       'Faixa etária / Entre 35 a 39', 'Água canalizada no domicílio / Não',
       'Faixa da renda total da família / Até 1 S.M.',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental EJA -séries iniciais (Supletivo 1ª a 4ª)',
       'Faixa etária / Entre 55 a 59', 'Recebe PBF família / Sim',
       'Função principal / Militar ou servidor público',
       'Curso mais elevado que a pessoa frequentou / Ensino Fundamental (duração 9 anos)',
       'Relação de parentesco com o responsável familiar / Cônjuge ou companheiro(a)',
       'Relação de parentesco com o responsável familiar / Pessoa Responsável pela Unidade Familiar - RF']

In [ ]:
x_tr = x_train[accepted]

In [ ]:
grid = {'learning_rate': [0.025, 0.05, 0.1, 0.2, 0.3],
        'max_depth': [2, 3, 5, 7, 10, 100],
        'colsample_bylevel': ['log2', 'sqrt', 0.25, 1.0],
        'subsample': [0.15, 0.5, 0.75, 1.0]}

clf = GridSearchCV(XGBRegressor(n_estimators=200), grid, cv = 10,
                   scoring='neg_root_mean_squared_error')
clf.fit(x_tr, y_train)
clf.best_params_

{'colsample_bylevel': 0.25,
 'learning_rate': 0.025,
 'max_depth': 5,
 'subsample': 1.0}

In [ ]:
xgb = XGBRegressor(n_estimators=200,colsample_bylevel=0.25,
                                learning_rate=0.025,max_depth=5,subsample=1.0
)

scores = cross_validate(xgb, x_tr, y_train, cv=10,
                               scoring='neg_root_mean_squared_error')

rmse = np.mean(scores['test_score']*(-1))
std = np.std(scores['test_score']*(-1))

print(rmse)
print(std)

4.874912567123582
0.532505988466458


## 2.5 LightGBM

In [ ]:
x_train.columns = [str(i) for i in list(range(len(x_train.columns)))]
y_train.columns = ['target']

In [ ]:
lgb = LGBMRegressor(n_estimators=200)

# no model selected default is Random Forest, if classification is False it is a Regression problem
Feature_Selector = BorutaShap(model=lgb,
                              importance_measure='shap',
                              classification=False)

Feature_Selector.fit(X=x_train, y=y_train, n_trials=100, random_state=0)

  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.154695 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 93902
[LightGBM] [Info] Number of data points in the train set: 3745, number of used features: 448
[LightGBM] [Info] Start training from score 10.097725
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026429 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 93798
[LightGBM] [Info] Number of data points in the train set: 3745, number of used features: 448
[LightGBM] [Info] Start training from score 10.097725
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026579 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 93822
[LightGBM] [Info] Number of data points in the train set: 3745, number of used features: 448
[LightGBM] [Info] Star

In [ ]:
accepted = Feature_Selector.Subset(tentative=True).columns
accepted

Index(['89', '83', '63', '162', '85', '55', '174', '210', '93', '160', '144',
       '154', '215', '20', '45', '48', '74', '201', '2', '87', '122', '98',
       '92', '164'],
      dtype='object')

In [ ]:
accepted = ['89', '83', '63', '162', '85', '55', '174', '210', '93', '160', '144',
       '154', '215', '20', '45', '48', '74', '201', '2', '87', '122', '98',
       '92', '164']

In [ ]:
x_tr = x_train[accepted]

In [ ]:
grid = {'learning_rate': [0.025, 0.05, 0.1, 0.2, 0.3],
        'num_leaves': [3, 7, 15, 31, 127, 1024],
        'top_rate': [0.2, 0.4, 0.6, 0.7],
        'other_rate': [0.05, 0.1, 0.3],
        'feature_fraction_bynode': ['log2', 'sqrt', 0.25, 1.0]}

clf = GridSearchCV(LGBMRegressor(n_estimators=200), grid, cv = 10,
                   scoring='neg_root_mean_squared_error')
clf.fit(x_tr, y_train)
clf.best_params_

A saída de streaming foi truncada nas últimas 5000 linhas.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

{'feature_fraction_bynode': 0.25,
 'learning_rate': 0.025,
 'num_leaves': 15,
 'other_rate': 0.05,
 'top_rate': 0.2}

In [ ]:
lgb = LGBMRegressor(n_estimators=200, feature_fraction_bynode=0.25,
                                learning_rate=0.025,num_leaves=15,
                                other_rate=0.05,top_rate=0.2
)

scores = cross_validate(lgb, x_tr, y_train, cv=10,
                               scoring='neg_root_mean_squared_error')

rmse = np.mean(scores['test_score']*(-1))
std = np.std(scores['test_score']*(-1))

print(rmse)
print(std)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009131 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5953
[LightGBM] [Info] Number of data points in the train set: 4815, number of used features: 24
[LightGBM] [Info] Start training from score 10.141807
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001989 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5877
[LightGBM] [Info] Number of data points in the train set: 4815, number of used features: 24
[LightGBM] [Info] Start training from score 10.052033
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5956
[LightGBM] [Info] Number of data points in the train set: 4815, number of used features: 24
[LightGBM] [Info] Start trai

In [ ]:
x_train = x_train[accepted]
x_test = x_test[accepted]

In [ ]:
def histogram_plot(mtx, width=2000, height=600):
  fig = px.imshow(mtx[:][-1:],
                #x=cadunico_data.columns,
                #y=cadunico_data.columns,
                text_auto=True,
                color_continuous_scale='Inferno',
                width=width, height=height
               )

  fig.update_layout(
      margin=dict(l=0, r=0, t=0, b=0),
      autosize=True,
      #xaxis_nticks=len(cadunico_data.columns),
      #yaxis_nticks=len(cadunico_data.columns)
      )

  fig.show()

In [ ]:
data = x_train.copy()
data['target'] = y_train
corr_matrix = data.corr()
histogram_plot(corr_matrix)

In [ ]:
spear_corr = data.corr(method="spearman")
histogram_plot(spear_corr)

# 3. Predição

In [ ]:
ctb = CatBoostRegressor(n_estimators = 200,
                                l2_leaf_reg=9, leaf_estimation_iterations=1,
                                learning_rate=0.05, max_depth=6)

ctb.fit(x_train, y_train)

0:	learn: 5.5390931	total: 219ms	remaining: 43.6s
1:	learn: 5.4961412	total: 295ms	remaining: 29.2s
2:	learn: 5.4551301	total: 401ms	remaining: 26.4s
3:	learn: 5.4141766	total: 512ms	remaining: 25.1s
4:	learn: 5.3808124	total: 617ms	remaining: 24.1s
5:	learn: 5.3466021	total: 723ms	remaining: 23.4s
6:	learn: 5.3117353	total: 838ms	remaining: 23.1s
7:	learn: 5.2805476	total: 956ms	remaining: 23s
8:	learn: 5.2532380	total: 1.08s	remaining: 22.9s
9:	learn: 5.2274150	total: 1.19s	remaining: 22.6s
10:	learn: 5.2042556	total: 1.31s	remaining: 22.5s
11:	learn: 5.1835855	total: 1.4s	remaining: 22s
12:	learn: 5.1628593	total: 1.5s	remaining: 21.5s
13:	learn: 5.1436113	total: 1.6s	remaining: 21.2s
14:	learn: 5.1242238	total: 1.72s	remaining: 21.2s
15:	learn: 5.1052543	total: 1.81s	remaining: 20.9s
16:	learn: 5.0884013	total: 1.92s	remaining: 20.7s
17:	learn: 5.0743612	total: 2.02s	remaining: 20.4s
18:	learn: 5.0589809	total: 2.11s	remaining: 20.1s
19:	learn: 5.0429232	total: 2.23s	remaining: 20s

In [ ]:
y_pred = ctb.predict(x_test)

# 4. Gráficos

In [ ]:
import plotly.graph_objects as go
import geopandas as gpd

In [ ]:
geojs = gpd.read_file('/content/drive/MyDrive/Geographic analisys/data/geojs-23-mun.json')
geojs['y_pred'] = y_pred
geojs['y_test'] = list(y_test)

In [ ]:
geojs

,id,name,description,geometry,y_pred,y_test
0,2300101,Abaiara,Abaiara,"POLYGON ((-38.96262 -7.40021, -38.96994 -7.425...",10.071949,12.57
1,2300150,Acarape,Acarape,"POLYGON ((-38.65581 -4.15804, -38.62384 -4.165...",11.719126,16.26
2,2300200,Acaraú,Acaraú,"POLYGON ((-40.31697 -2.80534, -40.26131 -2.810...",11.967423,10.23
3,2300309,Acopiara,Acopiara,"POLYGON ((-39.56759 -5.98316, -39.56142 -6.008...",10.722333,13.90
4,2300408,Aiuaba,Aiuaba,"POLYGON ((-40.10700 -6.35117, -40.08474 -6.368...",10.759420,11.78
...,...,...,...,...,...,...
179,2313807,Uruburetama,Uruburetama,"POLYGON ((-39.47198 -3.64954, -39.51421 -3.668...",11.744550,11.81
180,2313906,Uruoca,Uruoca,"POLYGON ((-40.69208 -3.24210, -40.66917 -3.247...",11.081930,11.59
181,2313955,Varjota,Varjota,"POLYGON ((-40.44576 -4.16348, -40.44220 -4.183...",10.580787,10.89
182,2314003,Várzea Alegre,Várzea Alegre,"POLYGON ((-39.21471 -6.59585, -39.20952 -6.607...",9.228829,10.12


In [ ]:
fig = px.choropleth_mapbox(geojs,
                           geojson=geojs.__geo_interface__,
                           locations=geojs.index,
                           color='y_test',
                           color_continuous_scale='Pinkyl',
                           mapbox_style='carto-positron',
                           zoom=6,
                           range_color=[0,40],
                           center = {"lat": -5.5, "lon": -43},
                           opacity=1.0,
                           labels={'y_test':'Real'})


#Remoção das margens
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
# Show the map
fig.show()


fig = px.choropleth_mapbox(geojs,
                           geojson=geojs.__geo_interface__,
                           locations=geojs.index,
                           color='y_pred',
                           color_continuous_scale='Pinkyl',
                           mapbox_style='carto-positron',
                           zoom=6,
                           range_color=[0,40],
                           center = {"lat": -5.5, "lon": -43},
                           opacity=1.0,
                           labels={'y_pred':'Predito'})


#Remoção das margens
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
# Show the map
fig.show()

In [ ]:
municipalities = list(df_test['Município'])

fig = go.Figure(data=[
    go.Bar(name='real', x=municipalities, y=list(y_test)),
    go.Bar(name='predicted', x=municipalities, y=y_pred)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [ ]:
print("RMSE: " + str(mean_squared_error(y_test, y_pred, squared=False)))
print("MAE: " + str(mean_absolute_error(y_test, y_pred)))
print("MAPE: " + str(mean_absolute_percentage_error(y_test, y_pred)))
print("R2: " + str(r2_score(y_test, y_pred)))

RMSE: 3.9513233303831186
MAE: 2.597476455936694
MAPE: 0.21762338846189894
R2: -0.0792094685478657


In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
sorted_feature_importance = ctb.feature_importances_.argsort()
plt.barh(x_train.columns[sorted_feature_importance],
        ctb.feature_importances_[sorted_feature_importance],
        color='black')
plt.xlabel("Importância das variáveis", fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=18)

In [ ]:
x_train.columns[sorted_feature_importance]

In [ ]:
ctb.feature_importances_[sorted_feature_importance]